# messages を扱う


## レッスンの目標
- messages API の形式を理解する
- モデルのレスポンスオブジェクトを扱い、理解する
- シンプルなマルチターンのチャットボットを作る


## 基本セットアップ
必要なパッケージを import し、クライアントオブジェクトを初期化します。
API キーの取得方法や安全な保存方法は、前回のチュートリアルを参照してください。


In [1]:
from dotenv import load_dotenv
from anthropic import Anthropic

# 環境変数を読み込む
load_dotenv()

# "ANTHROPIC_API_KEY" 環境変数を自動的に探して使う
client = Anthropic()


## messages の形式

前回のレッスンで見たように、`client.messages.create()` を使うと Claude にメッセージを送り、レスポンスを受け取れます：


In [4]:
response = client.messages.create(
    model="claude-sonnet-4-5-20250929",
    max_tokens=1000,
    messages=[
        {"role": "user", "content": "What flavors are used in Dr. Pepper? answer with Japanese."}
    ]
)

print(response.content[0].text)


ドクターペッパーの正確な配合は企業秘密ですが、一般的に以下のようなフレーバーが含まれていると言われています:

- チェリー（桜桃）
- リコリス（甘草）
- アーモンド
- バニラ
- プラム（李）
- オレンジ
- レモン
- コーラ
- ナツメグ
- シナモン
- その他のスパイスやフルーツのフレーバー

ドクターペッパーは23種類のフレーバーをブレンドしていると公式に言われていますが、全ての成分は公開されていません。


この部分をもう少し詳しく見てみましょう：

```python
messages=[
        {"role": "user", "content": "What flavors are used in Dr. Pepper?"}
    ]
```


`messages` パラメータは Claude API とやり取りする上で非常に重要です。会話の履歴やコンテキストを渡すことで、Claude が適切な応答を生成できるようになります。

`messages` は **メッセージ辞書のリスト**を受け取ります。各辞書は会話内の 1 つのメッセージを表します。
各メッセージ辞書には、次のキーが必要です：

- `role`: 送信者の役割を表す文字列。`"user"`（ユーザーの発話）または `"assistant"`（Claude の発話）。
- `content`: メッセージ本体。文字列、もしくはコンテンツ辞書のリストです。文字列を渡すと 1 つのテキストブロックとして扱われます。リストの場合は各辞書に `"type"`（例: `"text"` / `"image"`）と対応する内容を入れます。今のところ `content` は単純に文字列のままにします。

ユーザーメッセージ 1 つだけの例：

```python
messages = [
    {"role": "user", "content": "Hello Claude! How are you today?"}
]
```

複数メッセージ（会話）を渡す例：

```python
messages = [
    {"role": "user", "content": "Hello Claude! How are you today?"},
    {"role": "assistant", "content": "Hello! I'm doing well, thank you. How can I assist you today?"},
    {"role": "user", "content": "Can you tell me a fun fact about ferrets?"},
    {"role": "assistant", "content": "Sure! Did you know that excited ferrets make a clucking vocalization known as 'dooking'?"},
]
```

`messages` は **常に user と assistant が交互**になる点を覚えておいてください。

![alternating_messages](images/alternating_messages.png)


`messages` 形式を使うと、Claude への API 呼び出しを「会話」の形で構造化でき、**コンテキストを保持**できます。会話履歴（ユーザー/アシスタント双方）を丸ごと渡せるため、Claude は全体の文脈を踏まえて応答しやすくなり、より一貫性が高く関連性のある出力になりやすいです。

**注: 多くのユースケースでは会話履歴は不要です。`messages` が 1 件だけのリストでも、まったく問題ありません！**


***

## クイズ

各メッセージに必須のキー 2 つはどれ？

- **a)** `"sender"` と `"text"`
- **b)** `"role"` と `"content"`
- **c)** `"user"` と `"assistant"`
- **d)** `"input"` と `"output"`

<details>
  <summary>解答を見る</summary>

  **正解は b です。各メッセージには必ず `"role"` と `"content"` が必要です。**

</details>



***

## メッセージレスポンスを確認する
次に、Claude から返ってくるレスポンスの形（構造）を見てみましょう。

まずは Claude に簡単なことを頼みます：


In [5]:
response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    messages=[
        {"role": "user", "content": "Translate hello to French. Respond with a single word"}
    ]
)


では、返ってきた `response` の中身を確認します：


In [6]:
response


Message(id='msg_01YDbaxfnFawPe1RBwwiEDV1', content=[TextBlock(citations=None, text='Bonjour', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=19, output_tokens=7, server_tool_use=None, service_tier='standard'))

`Message` オブジェクトが返ってきて、いくつかのプロパティを持っています。例：

```
Message(id='msg_01Mq5gDnUmDESukTgwPV8xtG', content=[TextBlock(text='Bonjour.', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=19, output_tokens=8))
```

最も重要なのは `content` プロパティです。ここにモデルが生成した実際の内容が入ります。`content` は **コンテンツブロックのリスト**で、各ブロックは `type` によって形が決まります。

![message_content](images/message_content.png)

モデルの応答テキストを取り出すには次のようにします：


In [7]:
print(response.content[0].text)


Bonjour


`content` 以外にも `Message` には次のような情報が入っています：

- `id`: オブジェクトの一意な識別子
- `type`: オブジェクト種別（常に `"message"`）
- `role`: 生成されたメッセージの会話上の役割（常に `"assistant"`）
- `model`: リクエストを処理して応答を生成したモデル
- `stop_reason`: 生成が停止した理由（後で詳しく扱います）
- `stop_sequence`: これも後ほど扱います
- `usage`: 課金やレート制限に関する情報
  - `input_tokens`: 入力トークン数
  - `output_tokens`: 出力トークン数

これらにアクセスできることは重要ですが、もし 1 つだけ覚えるならこれです：**`content` にモデルが生成した実体が入っています。**


***
## 演習

`translate` という関数を書いてみましょう。引数は 2 つです：
- 単語（word）
- 言語（language）

`translate` を呼び出すと、Claude に `word` を `language` に翻訳させた結果を返すようにしてください。例：

```python
translate("hello", "Spanish")
# 'The word "hello" translated into Spanish is: Hola'

translate("chicken", "Italian")
# 'The Italian word for "chicken" is: pollo'
```

おまけ：プロンプトを工夫して、前置きなしで翻訳語だけ返すようにできると最高です：

```python
translate("chicken", "Italian")
# 'pollo'
```


<details>
  <summary>解答例を見る</summary>

  解答例の 1 つ：

  ```python
  def translate(word, language):
    response = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1000,
        messages=[
            {"role": "user", "content": f"Translate the word {word} into {language}.  Only respond with the translated word, nothing else"}
        ]
    )
    return response.content[0].text
  ```

</details>



In [10]:
def translate(word, language):
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        messages=[
            {"role": "user", "content": f"Translate the word {word} into {language}.  Only respond with the translated word, nothing else"}
        ]
    )
    return response.content[0].text

In [13]:
translate("hello", "Spanish")

'Hola'

In [14]:
translate("chicken", "Italian")

'pollo'

***

## messages リストでよくあるミス

### ミス #1: assistant メッセージから始めてしまう


慣れないうちは `messages` リストの扱いでミスしがちです。`messages` は **必ず `user` メッセージから始める必要**があります。
次のコードは `assistant` メッセージから始まっているためエラーになります：


In [ ]:
response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    messages=[
        {"role": "assistant", "content": "Hello there!"}
    ]
)
print(response.content[0].text)


### ミス #2: 交互になっていない

`messages` は `user` と `assistant` が交互でなければなりません。このルールに従わないとエラーになります：


In [ ]:
response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    messages=[
        {"role": "user", "content": "Hey there!"},
        {"role": "assistant", "content": "Hi there!"},
        {"role": "assistant", "content": "How can I help you??"}
    ]
)
print(response.content[0].text)


## messages リストの使いどころ


### Claude に言わせたい言葉を先に置く（"put words in Claude's mouth"）

出力をかなり厳密にコントロールしたいときによく使われるのが、「Claude の口に言葉を入れる」戦略です。`user` メッセージだけでなく、Claude が出力生成に利用する `assistant` メッセージをこちらから与えることもできます。

Anthropic の API では `user` メッセージだけに限定されません。`assistant` メッセージを渡すと、Claude は **最後の `assistant` トークンの続き**として会話を継続します。ただし、会話は必ず `user` メッセージから開始する必要があります。

たとえば「calming mountain air」という 1 行目から始まる俳句を書かせたいとします。その場合、次の会話履歴を渡せます：

```python
messages=[
        {"role": "user", "content": f"Generate a beautiful haiku"},
        {"role": "assistant", "content": "calming mountain air"}
    ]
```

ここでは「俳句を生成して」と伝えた上で、俳句の 1 行目を Claude の口に入れています。


In [15]:
response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=500,
    messages=[
        {"role": "user", "content": f"Generate a beautiful haiku"},
        {"role": "assistant", "content": "calming mountain air"}
    ]
)
print(response.content[0].text)


,
petals drift on gentle breeze,
tranquil moment's peace.


渡した 1 行目を先頭に付けて、俳句全体を表示するには：


In [16]:
print("calming mountain air" + response.content[0].text)


calming mountain air,
petals drift on gentle breeze,
tranquil moment's peace.


### Few-shot prompting（少数例で誘導する）

有用なプロンプト戦略の 1 つに **few-shot prompting** があります。これは、モデルに少数の **例（examples）** を見せて出力を誘導する方法です。`messages` の会話履歴は、Claude に例を渡すのにとても向いています。

たとえば、Claude にツイートの感情（sentiment）分析をさせたいとします。まずは単純に「このツイートの感情を分析して」と頼んで、どんな出力になるか見てみます：


In [17]:
response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=500,
    messages=[
        {"role": "user", "content": f"Analyze the sentiment in this tweet: Just tried the new spicy pickles from @PickleCo, and my taste buds are doing a happy dance! 🌶️🥒 #pickleslove #spicyfood"},
    ]
)
print(response.content[0].text)


Based on the analysis of the tweet, the sentiment expressed is positive. Some key points that indicate this:

1. The use of an exclamation mark and positive emojis (e.g., 🌶️🥒) suggest the person is excited and pleased with the new spicy pickles from PickleCo.

2. The phrase "my taste buds are doing a happy dance!" conveys a sense of enjoyment and satisfaction with the product.

3. The use of hashtags like "#pickleslove" and "#spicyfood" further reinforces the positive sentiment towards the spicy pickles.

Overall, the tweet expresses a very favorable and enthusiastic sentiment about the new spicy pickles, indicating the person had a positive experience trying them.


最初に上のコードを実行したとき、Claude は次のように長い回答を返しました：

```
The sentiment in this tweet is overwhelmingly positive. The user expresses their enjoyment of the new spicy pickles from @PickleCo, using enthusiastic language and emojis to convey their delight.

Positive indicators:
1. "My taste buds are doing a happy dance!" - This phrase indicates that the user is extremely pleased with the taste of the pickles, to the point of eliciting a joyful physical response.

2. Emojis - The use of the hot pepper 🌶️ and cucumber 🥒 emojis further emphasizes the user's excitement about the spicy pickles.

3. Hashtags - The inclusion of #pickleslove and #spicyfood hashtags suggests that the user has a strong affinity for pickles and spicy food, and the new product aligns perfectly with their preferences.

4. Exclamation mark - The exclamation mark at the end of the first sentence adds emphasis to the user's positive experience.

Overall, the tweet conveys a strong sense of satisfaction, excitement, and enjoyment related to trying the new spicy pickles from @PickleCo.
```


これは良い回答ですが、特に大量のツイートを自動で感情分析したい場合、必要以上に情報量が多いかもしれません。

たとえば、単語（POSITIVE / NEUTRAL / NEGATIVE）のような標準化された形式や、数値（1, 0, -1）のような形式で返してほしいこともあります。ここでは読みやすさと単純さのため、"POSITIVE" か "NEGATIVE" のどちらかで返すようにしてみましょう。

その方法の 1 つが few-shot prompting です。Claude に「こう答えてほしい」という例を会話履歴で見せます：

```python
messages=[
        {"role": "user", "content": "Unpopular opinion: Pickles are disgusting. Don't @ me"},
        {"role": "assistant", "content": "NEGATIVE"},
        {"role": "user", "content": "I think my love for pickles might be getting out of hand. I just bought a pickle-shaped pool float"},
        {"role": "assistant", "content": "POSITIVE"},
        {"role": "user", "content": "Seriously why would anyone ever eat a pickle?  Those things are nasty!"},
        {"role": "assistant", "content": "NEGATIVE"},
        {"role": "user", "content": "Just tried the new spicy pickles from @PickleCo, and my taste buds are doing a happy dance! 🌶️🥒 #pickleslove #spicyfood"},
    ]
```


In [19]:
response = client.messages.create(
    model="claude-sonnet-4-5-20250929",
    max_tokens=500,
    messages=[
        {"role": "user", "content": "Unpopular opinion: Pickles are disgusting. Don't @ me"},
        {"role": "assistant", "content": "NEGATIVE"},
        {"role": "user", "content": "I think my love for pickles might be getting out of hand. I just bought a pickle-shaped pool float"},
        {"role": "assistant", "content": "POSITIVE"},
        {"role": "user", "content": "Seriously why would anyone ever eat a pickle?  Those things are nasty!"},
        {"role": "assistant", "content": "NEGATIVE"},
        {"role": "user", "content": "Just tried the new spicy pickles from @PickleCo, and my taste buds are doing a happy dance! 🌶️🥒 #pickleslove #spicyfood"},
    ]
)
print(response.content[0].text)


POSITIVE


***

## 演習

### 課題: チャットボットを作る

シンプルなマルチターンのコマンドライン・チャットボット（スクリプト）を作ってください。`messages` 形式はチャットベースのアプリを作るのに向いています。Claude でチャットボットを作るのは次の通りシンプルです：

1. 会話履歴を保存するためのリストを用意する
2. `input()` でユーザーからメッセージを受け取り、`messages` リストに追加する
3. 会話履歴を Claude に送る
4. Claude の応答を表示する
5. Claude の `assistant` 応答を履歴に追加する
6. 手順 2 に戻って繰り返す（ループにして、終了できる方法も用意しましょう）


<details>
    <summary>解答例を見る</summary>

```python

conversation_history = []

while True:
    user_input = input("User: ")

    if user_input.lower() == "quit":
        print("Conversation ended.")
        break

    conversation_history.append({"role": "user", "content": user_input})

    response = client.messages.create(
        model="claude-3-haiku-20240307",
        messages=conversation_history,
        max_tokens=500
    )

    assistant_response = response.content[0].text
    print(f"Assistant: {assistant_response}")
    conversation_history.append({"role": "assistant", "content": assistant_response})
```

</details>


***
